<a href="https://colab.research.google.com/github/aderdouri/ActuarialThesis/blob/master/Notebooks/Regressors/myRegressorsCAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cloner la branche contenant le dataset et le code qui va avec.

In [1]:
!rm -rf ActuarialThesis
!git clone https://github.com/aderdouri/ActuarialThesis.git
%ls -ltr ActuarialThesis

Cloning into 'ActuarialThesis'...
remote: Enumerating objects: 1115, done.
remote: Counting objects: 100% (398/398), done.
remote: Compressing objects: 100% (212/212), done.
remote: Total 1115 (delta 216), reused 347 (delta 184), pack-reused 717
Receiving objects: 100% (1115/1115), 134.07 MiB | 3.68 MiB/s, done.
Resolving deltas: 100% (506/506), done.
total 24
drwxr-xr-x 2 root root 4096 Nov 15 13:02 Data/
drwxr-xr-x 2 root root 4096 Nov 15 13:02 EMTboost/
drwxr-xr-x 6 root root 4096 Nov 15 13:02 Notebooks/
drwxr-xr-x 2 root root 4096 Nov 15 13:02 R/
drwxr-xr-x 2 root root 4096 Nov 15 13:02 src/
-rw-r--r-- 1 root root   54 Nov 15 13:02 README.md


In [2]:
!mkdir ActuarialThesis/plots_rgr
!ls -ltr ActuarialThesis/plots_rgr

total 0


In [3]:
# Ajouter le répértoire src
import sys
sys.path.insert(0,'./ActuarialThesis/src/')

In [4]:
import installHelper

In [5]:
# Installer les packages nécéssaires
%time installHelper.installALL()

CPU times: user 585 ms, sys: 80.4 ms, total: 665 ms
Wall time: 2min 15s


In [6]:
# On doit trouver tous les packages mentionés dans le grep
!pip list -v | grep -e catboost -e 'imbalanced-learn' -e 'optuna' -e 'catboost' -e 'lime' -e 'shap'

catboost                         1.2.2                 /usr/local/lib/python3.10/dist-packages pip
imbalanced-learn                 0.10.1                /usr/local/lib/python3.10/dist-packages pip
lime                             0.2.0.1               /usr/local/lib/python3.10/dist-packages pip
optuna                           3.4.0                 /usr/local/lib/python3.10/dist-packages pip
shap                             0.43.0                /usr/local/lib/python3.10/dist-packages pip
shapely                          2.0.2                 /usr/local/lib/python3.10/dist-packages pip


In [7]:
# Importer les packages nécéssaires
from helper import *

In [8]:
# Appliquer le theme par défaut
sns.set_theme()

In [9]:
!ls -ltr ActuarialThesis/Data

total 46044
-rw-r--r-- 1 root root   562212 Nov 15 13:02 BASEAUTO.csv
-rw-r--r-- 1 root root   201188 Nov 15 13:02 df_models.csv
-rw-r--r-- 1 root root  3263186 Nov 15 13:02 encodedBASEAUTO.csv
-rw-r--r-- 1 root root  3273326 Nov 15 13:02 encodedCategoricalBASEAUTO.csv
-rw-r--r-- 1 root root  2236112 Nov 15 13:02 encodedTDboostEMTDboost.csv
-rw-r--r-- 1 root root   698687 Nov 15 13:02 encodedTestBASEAUTO.csv
-rw-r--r-- 1 root root  2792067 Nov 15 13:02 encodedTrainBASEAUTO.csv
-rw-r--r-- 1 root root 31528920 Nov 15 13:02 freMTPL2freq.csv
-rw-r--r-- 1 root root   403127 Nov 15 13:02 freMTPL2sev.csv
-rw-r--r-- 1 root root   846739 Nov 15 13:02 transformedBASEAUTO.csv
-rw-r--r-- 1 root root   580225 Nov 15 13:02 transformedCatBASEAUTO.csv
-rw-r--r-- 1 root root   738484 Nov 15 13:02 transformedCategorialBASEAUTO.csv


In [10]:
df = pd.read_csv('ActuarialThesis/Data/transformedCategorialBASEAUTO.csv')
df.head()

,SEX,STATUT,CSP,USAGE,K8000,RM,CAR,CLA,ALI,ENE,VIT,SEGM,CHARGE,GARAGE,AGECOND_CAT,ACV_CAT
0,F,C,50,Prive_Trajet,N,64,BER,A,CAR,ES,130-140,A,0.0,Autre,35_45,>=10
1,F,A,50,Prive,N,50,BER,A,CAR,ES,001-130,0,0.0,Autre,55_70,>=10
2,F,C,AUTRE,Prive_Trajet,N,95,BER,A,CAR,ES,001-130,0,0.0,Autre,<=21,>=10
3,F,A,50,Prive,N,50,BER,A,CAR,ES,001-130,0,0.0,Autre,55_70,>=10
4,F,A,50,Prive,N,95,BER,A,CAR,ES,140-150,B,0.0,Autre,25_35,>=10


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10496 entries, 0 to 10495
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   SEX          10496 non-null  object 
 1   STATUT       10496 non-null  object 
 2   CSP          10496 non-null  object 
 3   USAGE        10496 non-null  object 
 4   K8000        10496 non-null  object 
 5   RM           10496 non-null  int64  
 6   CAR          10496 non-null  object 
 7   CLA          10496 non-null  object 
 8   ALI          10496 non-null  object 
 9   ENE          10496 non-null  object 
 10  VIT          10496 non-null  object 
 11  SEGM         10496 non-null  object 
 12  CHARGE       10496 non-null  float64
 13  GARAGE       10496 non-null  object 
 14  AGECOND_CAT  10496 non-null  object 
 15  ACV_CAT      10496 non-null  object 
dtypes: float64(1), int64(1), object(14)
memory usage: 1.3+ MB


In [12]:
# Supprimer les gros sinistres
df = df[df['CHARGE']<=5000]

## Partir du dataset déjà encodé.

In [13]:
X = df.drop('CHARGE', axis=1)
y = df['CHARGE']

# Split the train set into Train/Test sets: 80:20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

In [14]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((8327, 15), (8327,), (2082, 15), (2082,))

In [15]:
X_train.columns

Index(['SEX', 'STATUT', 'CSP', 'USAGE', 'K8000', 'RM', 'CAR', 'CLA', 'ALI',
       'ENE', 'VIT', 'SEGM', 'GARAGE', 'AGECOND_CAT', 'ACV_CAT'],
      dtype='object')

## CatBoostRegressor

In [16]:
cat_features = ['SEX', 'STATUT', 'CSP', 'USAGE', 'K8000', 'CAR', 'CLA', 'ALI', 'ENE', 'VIT', 'SEGM', 'GARAGE', 'AGECOND_CAT', 'ACV_CAT']

In [17]:
train_pool = Pool(X_train, label=y_train,
                  cat_features=cat_features)
test_pool = Pool(X_test, label=y_test,
                 cat_features=cat_features)

#cb_tweedie = CatBoostRegressor(loss_function='Tweedie:variance_power=1.9', n_estimators=500, silent=True)
#cb_tweedie.fit(train_pool, eval_set=test_pool)

#cb_rmse = CatBoostRegressor(loss_function='RMSE', n_estimators=500, silent=True)
#cb_rmse.fit(train_pool, eval_set=test_pool)

In [ ]:
from sklearn.model_selection import GridSearchCV

model = CatBoostRegressor(n_estimators=1000, silent=True)
parameters = {
    'learning_rate': [0.001, 0.01, 0.02, 0.03, 0.1],
    'depth': [2, 4, 6],
    'l2_leaf_reg': [3, 5, 7, 9],
    'loss_function' : ['Tweedie:variance_power='+str(var_power) for var_power in np.linspace(1.01, 1.999, 10)]
    }

grid = GridSearchCV(estimator=model, param_grid = parameters, cv = 2, n_jobs=-1)
grid.fit(X_train, y_train, cat_features=cat_features)

In [ ]:
grid.best_params_

In [ ]:
from catboost import Pool

#train_pool = Pool(X_train, label = y_train, cat_features=cat_features)
#test_pool = Pool(X_test, label = y_test, cat_features=cat_features)

cb_tweedie = CatBoostRegressor(n_estimators=1000,
                               depth=2,
                               l2_leaf_reg=9,
                               learning_rate=0.01,
                               #use_best_model=True,
                               loss_function='Tweedie:variance_power=1.6693333333333333',
                               silent=True)
cb_tweedie.fit(X_train, y_train, eval_set=test_pool, cat_features=cat_features)

#cb_rmse = CatBoostRegressor(n_estimators=1000, loss_function='RMSE', silent=True)
#cb_rmse.fit(train_pool, eval_set=test_pool, cat_features=cat_features)

In [ ]:
#y_pred_rmse_tweedie = np.clip(cb_rmse.predict(test_pool), 0, None)
y_pred_cb_tweedie = cb_tweedie.predict(test_pool)
print('MSLE score:')
print('\tCatBoost tweedie loss\t', mean_squared_log_error(y_test, y_pred_cb_tweedie))

In [ ]:
# Mean squared logarithmic error (MSLE)
#from sklearn.metrics import mean_squared_log_error

In [ ]:
df_ObsVsPred = df_ObservedVsPredicted(y_test, y_pred_cb_tweedie)
fig, ax = plotLiftChart(df_ObsVsPred)
fig.savefig('./ActuarialThesis/plots_rgr/myCatBoostRegressorLiftChart.pdf')

In [ ]:
df_ObsVsPred = df_ObservedVsPredicted(y_test, y_pred_cb_tweedie)
fig, ax = plotLorenzCurve(df_ObsVsPred)
fig.savefig('./ActuarialThesis/plots_rgr/myCatBoostRegressorLorenzCurve.pdf')

## LightGBMRegressor

In [ ]:
X = df.drop('CHARGE', axis=1)
y = df['CHARGE']

# Split the train set into Train/Test sets: 80:20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=13)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=13)

In [ ]:
EARLY_STOPPING_ROUND = 100

In [ ]:
for c in cat_features:
    X_train[c] = X_train[c].astype('category')
    X_val[c] = X_val[c].astype('category')
    X_test[c] = X_test[c].astype('category')

In [ ]:
from sklearn.model_selection import GridSearchCV

model = LGBMRegressor(n_estimators=1000, verbose=-1)
parameters = {
    'learning_rate': [0.001, 0.01, 0.02, 0.03, 0.1],
    'max_depth': [2, 4, 6],
    #'reg_lambda': [3, 5, 7, 9],
    'objective' : ['tweedie_variance_power='+str(var_power) for var_power in np.linspace(1.01, 1.999, 10)]
    }

grid = GridSearchCV(estimator=model, param_grid = parameters, cv = 2, n_jobs=-1)
grid.fit(X_train, y_train)

In [ ]:
grid.best_params_

In [ ]:
lgbm_rgr = LGBMRegressor(objective='tweedie_variance_power=1.03',
                         n_estimators=1000,
                         learning_rate=0.001,
                         max_depth=2,
                         random_state=13,
                         verbose=-1)

lgbm_rgr.fit(X_train, y_train,
             eval_set=[(X_val, y_val)],
             eval_metric='rmse',
             callbacks=[early_stopping(stopping_rounds=EARLY_STOPPING_ROUND),
                        #log_evaluation(period=20, show_stdv=True),
             ]
        )
y_pred_lgbm_rgr = lgbm_rgr.predict(X_val)
rmse = mean_squared_error(y_val, y_pred_lgbm_rgr, squared=False)
print("RMSE: %.5f" % rmse)

In [ ]:
#y_pred_rmse_tweedie = np.clip(cb_rmse.predict(test_pool), 0, None)
y_pred = lgbm_rgr.predict(X_test)
print('MSLE score:')
print('\tCatBoost tweedie loss\t', mean_squared_log_error(y_test, y_pred))

In [ ]:
df_ObsVsPred = df_ObservedVsPredicted(y_test, y_pred)
fig, ax = plotLiftChart(df_ObsVsPred)
fig.savefig('./ActuarialThesis/plots_rgr/myCatBoostRegressorLiftChart.pdf')

In [ ]:
df_ObsVsPred = df_ObservedVsPredicted(y_test, y_pred)
fig, ax = plotLorenzCurve(df_ObsVsPred)
fig.savefig('./ActuarialThesis/plots_rgr/myCatBoostRegressorLorenzCurve.pdf')

In [ ]:
from sklearn.model_selection import GridSearchCV

model = XGBRegressor(n_estimators=1000, enable_categorical=True, silent=True)
parameters = {
    'objective' : ['reg:tweedie'],
    'learning_rate': [0.001, 0.01, 0.02, 0.1],
    'depth': [2, 4, 6],
    #'l2_leaf_reg': [3, 5, 7, 9],
    'subsample': [0.8, 0.9],
    'tweedie_variance_power' : np.linspace(1.01, 1.999, 10),
    }

grid = GridSearchCV(estimator=model, param_grid = parameters, cv = 2, n_jobs=-1)
grid.fit(X_train, y_train)

In [ ]:
grid.best_params_

In [ ]:
xgb_rgr = XGBRegressor(objective='reg:tweedie',
                       tweedie_variance_power=1.2297777777777779,
                       n_estimators=1000,
                       learning_rate=0.001,
                       max_depth=2,
                       subsample = 0.8,
                       #early_stopping_rounds=50,
                       enable_categorical=True,
                       random_state=13
                       )

xgb_rgr.fit(X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=False)

y_pred_xgb_rgr = xgb_rgr.predict(X_test)
rmse = mean_squared_error(y_test, y_pred_xgb_rgr, squared=False)
print("RMSE: %.5f" % rmse)

In [ ]:
#y_pred_rmse_tweedie = np.clip(cb_rmse.predict(test_pool), 0, None)
y_pred = xgb_rgr.predict(X_test)
print('MSLE score:')
print('\tCatBoost tweedie loss\t', mean_squared_log_error(y_test, y_pred))

In [ ]:
df_ObsVsPred = df_ObservedVsPredicted(y_test, y_pred)
fig, ax = plotLiftChart(df_ObsVsPred)
fig.savefig('./ActuarialThesis/plots_rgr/myCatBoostRegressorLiftChart.pdf')

In [ ]:
df_ObsVsPred = df_ObservedVsPredicted(y_test, y_pred)
fig, ax = plotLorenzCurve(df_ObsVsPred)
fig.savefig('./ActuarialThesis/plots_rgr/myCatBoostRegressorLorenzCurve.pdf')

In [ ]:
import xgboost as xgb
# Get a graph
graph = xgb.to_graphviz(xgb_rgr, num_trees=1)
# Or get a matplotlib axis
ax = xgb.plot_tree(xgb_rgr, num_trees=1)
# Get feature importances
xgb_rgr.feature_importances_